<a href="https://colab.research.google.com/github/suvaline/DeepLearningComparison/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
from __future__ import print_function
import keras
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from keras.datasets import mnist, cifar10, cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 10
dataset_name = "mnist" 
max_pooling = False
dropout = False
# input image dimensions
if(dataset_name == "mnist"):
  img_rows, img_cols = 28, 28
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  color_count = 1
if(dataset_name == "cifar10"):
  img_rows, img_cols = 32, 32
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()
  color_count = 3
if(dataset_name == "cifar100"):
  num_classes = 100
  img_rows, img_cols = 32, 32
  (x_train, y_train), (x_test, y_test) = cifar100.load_data()
  color_count = 3
# the data, split between train and test sets



11493376/11490434 [==============================] - 1s 0us/step


In [0]:

def create_2d_array(x,y):
  a = [0] * x
  for i in range(x):
    a[i] = [0] * y


In [4]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], color_count, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], color_count, img_rows, img_cols)
    input_shape = (color_count, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, color_count)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, color_count)
    input_shape = (img_rows, img_cols, color_count)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


def cnn(depth, width, input_shape, num_classes, max_pooling = False, dropout = False, kernel_size = (3, 3)):
  model = Sequential()
  model.add(Conv2D(width, kernel_size, activation='relu', input_shape = input_shape))
  
  i = 0
  while i<depth-1:
    model.add(Conv2D(width, kernel_size, activation='relu'))
    if max_pooling:
      model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout:
      model.add(Dropout(0.4))
    i += 1
  model.add(Flatten())
  model.add(Dense(num_classes, activation='softmax'))
  return model



x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
def create_2d_array(x,y):
  a = [0] * x
  for i in range(x):
    a[i] = [0] * y
  return a
def plot(history):
  plt.figure(figsize=[8,6])
  plt.plot(history.history['loss'],'r',linewidth=3.0)
  plt.plot(history.history['val_loss'],'b',linewidth=3.0)
  plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
  plt.xlabel('Epochs ',fontsize=16)
  plt.ylabel('Loss',fontsize=16)
  plt.title('Loss Curves',fontsize=16)
  plt.show()
 
#Plot the Accuracy Curves
  plt.figure(figsize=[8,6])
  plt.plot(history.history['acc'],'r',linewidth=3.0)
  plt.plot(history.history['val_acc'],'b',linewidth=3.0)
  plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
  plt.xlabel('Epochs ',fontsize=16)
  plt.ylabel('Accuracy',fontsize=16)
  plt.title('Accuracy Curves',fontsize=16)
  plt.show()

def create_heatmaps(rows, columns, array, create_max_heatmap = 'false'):
  print("mean heatmap:")
  mean_array = np.mean(array, axis=2)
  ax = sns.heatmap(mean_array, linewidth=0.5,vmin=0, vmax=1, annot=True)
  plt.show()
  last_loss_array = create_2d_array(rows, columns)
  for i in range(rows):
    for j in range(columns):
      last_loss_array[i][j] = array[i][j][epochs-1]
  
  print("last result heatmap:")
  ax = sns.heatmap(last_loss_array, linewidth=0.5,vmin=0, vmax=1, annot=True)
  plt.show()
  if create_max_heatmap == 'true':
    print("max heatmap :")
    max_array = np.amax(array, axis=2)
    ax = sns.heatmap(max_array, linewidth=0.5,vmin=0, vmax=1, annot=True)
    plt.show()

def show_heatmaps(arrays, depth, width):
  loss_array = arrays[0]
  val_loss_array = arrays[1]
  acc_array = arrays[2]
  val_acc_array = arrays[3]
  print("loss heatmaps :")
  create_heatmaps(depth, width, loss_array)
  print("validation loss heatmaps :")
  create_heatmaps(depth, width, val_loss_array)
  print("accuracy heatmaps :")
  create_heatmaps(depth, width, acc_array, create_max_heatmap = 'true')
  print("validation accuracy heatmaps :")
  create_heatmaps(depth, width, val_acc_array, create_max_heatmap = 'true')
def generate_train_and_plot(depth, multiplier, arrays, max_depth, width_multiplier):
  #depth = 0
  #loss_array = create_2d_array(max_depth, width_multiplier)
  #val_loss_array = create_2d_array(max_depth, width_multiplier)
  #acc_array = create_2d_array(max_depth, width_multiplier)
  #val_acc_array = create_2d_array(max_depth, width_multiplier)
  original_multiplier = multiplier
  loss_array = arrays[0]
  val_loss_array = arrays[1]
  acc_array = arrays[2]
  val_acc_array = arrays[3]
  while depth < max_depth:
    multiplier = original_multiplier
    while multiplier < width_multiplier:
      model = cnn(depth*2, 5 + 40 * multiplier, input_shape, num_classes, max_pooling = max_pooling, dropout = dropout)
      model.compile(loss=keras.losses.categorical_crossentropy,
                    optimizer=keras.optimizers.Adadelta(),
                    metrics=['accuracy'])
      model.summary()
      history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
      print("depth : ", depth, "multiplier", multiplier)
      loss_array[depth][multiplier] = history.history['loss']
      print("loss_array =", loss_array)
      val_loss_array[depth][multiplier] = history.history['val_loss']
      print("val_loss_array =", val_loss_array)
      acc_array[depth][multiplier] = history.history['acc']
      print("accuracy_array =", acc_array)
      val_acc_array[depth][multiplier] = history.history['val_acc']
      print("val_accuracy_array =", val_acc_array)
      multiplier += 1
      keras.backend.clear_session()
    depth += 1
    print("depth : ", depth)
  arrays = [loss_array, val_loss_array, acc_array, val_acc_array]
  return arrays
  


In [6]:
max_depth = 5
width_multiplier = 5
arrays = [create_2d_array(max_depth, width_multiplier), create_2d_array(max_depth, width_multiplier), create_2d_array(max_depth, width_multiplier), create_2d_array(max_depth, width_multiplier)]
arrays = generate_train_and_plot(0,0,arrays,max_depth,width_multiplier)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 5)         50        
_________________________________________________________________
flatten_1 (Flatten)          (None, 3380)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                33810     
Total params: 33,860
Trainable params: 33,860
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 121us/step - loss: 0.3962 - acc: 0.8884 - val_loss: 0.2476 - val_acc: 0.9330
Epoch 2/10
60000/60000 [==============================] - 2s 31us/step - loss: 0.2236 - acc: 0

KeyboardInterrupt: ignored

In [0]:
show_heatmaps(arrays, max_depth, width_multiplier)